In [1]:
%matplotlib inline

from datetime import datetime

import datacube
import utils.dc_utilities as utils

from utils.dc_mosaic import create_mosaic
from utils.dc_water_classifier import *

import warnings; warnings.simplefilter('ignore')

In [2]:
start_time = datetime.now()
print("Start time: " + str(start_time))

Start time: 2017-03-13 14:15:08.929633


In [3]:
# Initialize data cube object
dc = datacube.Datacube(app='dc-coastal-erosion')

# Set query parameters
platform        = 'LANDSAT_7'
product_type    = 'ls7_ledaps_togo'

min_lon         = 1.1919
max_lon         = 1.4509
min_lat         = 6.0809
max_lat         = 6.2218



In [27]:
range(2000, 2015)

range(2000, 2015)

In [28]:
start_date      = datetime.strptime('2007-01-01', '%Y-%m-%d')
end_date        = datetime.strptime('2007-12-31', '%Y-%m-%d')

In [29]:
# Retrieve data from Data Cube
dataset_in = dc.load(platform=platform,
                     product=product_type,
                     time=(start_date, end_date),
                     lon=(min_lon, max_lon),
                     lat=(min_lat, max_lat))

In [30]:
mosaic = create_mosaic(dataset_in)

In [31]:
mosaic

<xarray.Dataset>
Dimensions:            (latitude: 520, longitude: 957)
Coordinates:
  * latitude           (latitude) float64 6.222 6.222 6.221 6.221 6.221 6.22 ...
  * longitude          (longitude) float64 1.192 1.192 1.192 1.193 1.193 ...
Data variables:
    blue               (latitude, longitude) int16 1468 1496 1468 1468 1468 ...
    green              (latitude, longitude) int16 1616 1647 1616 1616 1647 ...
    red                (latitude, longitude) int16 1750 1750 1750 1750 1777 ...
    nir                (latitude, longitude) int16 2273 2273 2273 2273 2273 ...
    swir1              (latitude, longitude) int16 2697 2553 2517 2517 2553 ...
    swir2              (latitude, longitude) int16 2361 2173 2248 2173 2361 ...
    cf_mask            (latitude, longitude) int16 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
    cf_mask_conf       (latitude, longitude) int16 1 1 1 1 2 2 2 2 1 1 1 1 1 ...
    adjacent_cloud_qa  (latitude, longitude) int16 255 255 255 255 255 255 ...
    atmos_opacity   

In [32]:
water_class = wofs_classify(mosaic, mosaic=True)
water_class.wofs.values[water_class.wofs.values == -9999] = np.nan

In [33]:
wofs = water_class.wofs.values

In [34]:
wofs[mosaic.swir2.values > 100] = np.nan

In [35]:
coastline = np.zeros(wofs.shape)

for i in range(wofs.shape[0]):
    for j in range(wofs.shape[1]):
        pixel = wofs[i,j]
        if pixel == 0 and np.nansum(wofs[i-1:i+2, j-1:j+2]) >= 1 and np.nansum(wofs[i-1:i+2, j-1:j+2]) <= 5:
            coastline[i,j] = 1

In [36]:
len(water_class.latitude)

520

In [37]:
wofs[np.isnan(wofs)] = -9999

In [38]:
wofs

array([[ -9.99900000e+03,  -9.99900000e+03,  -9.99900000e+03, ...,
         -9.99900000e+03,  -9.99900000e+03,  -9.99900000e+03],
       [ -9.99900000e+03,  -9.99900000e+03,  -9.99900000e+03, ...,
         -9.99900000e+03,  -9.99900000e+03,  -9.99900000e+03],
       [ -9.99900000e+03,  -9.99900000e+03,  -9.99900000e+03, ...,
         -9.99900000e+03,  -9.99900000e+03,  -9.99900000e+03],
       ..., 
       [ -9.99900000e+03,  -9.99900000e+03,  -9.99900000e+03, ...,
         -9.99900000e+03,  -9.99900000e+03,  -9.99900000e+03],
       [ -9.99900000e+03,  -9.99900000e+03,  -9.99900000e+03, ...,
         -9.99900000e+03,  -9.99900000e+03,   1.00000000e+00],
       [ -9.99900000e+03,  -9.99900000e+03,  -9.99900000e+03, ...,
         -9.99900000e+03,  -9.99900000e+03,  -9.99900000e+03]])

In [39]:


driver = gdal.GetDriverByName('GTiff')
raster = driver.Create('2007_wofs.tif', len(water_class.longitude), len(water_class.latitude), 1, gdal.GDT_Float32)

crs = dataset_in.crs
spatial_ref = utilities.get_spatial_ref(crs)

# Upper left coordinates
ul_lon = dataset_in.longitude.values[0]
ul_lat = dataset_in.latitude.values[0]

# Resolution
products = dc.list_products()
resolution = products.resolution[products.name == 'ls7_ledaps_togo']
lon_dist = resolution.values[0][1]
lat_dist = resolution.values[0][0]

# Rotation
lon_rtn = 0
lat_rtn = 0

geotransform = (ul_lon, lon_dist, lon_rtn, ul_lat, lat_rtn, lat_dist)

raster.SetGeoTransform(geotransform)
raster.SetProjection(spatial_ref)

out_band = raster.GetRasterBand(1)
out_band.SetNoDataValue(-9999)
out_band.WriteArray(wofs)
out_band.FlushCache()

raster.FlushCache()
out_band = None
raster = None

In [40]:
end_time = datetime.now()
print("Elapsed time: " + str(end_time - start_time))

Elapsed time: 0:00:15.342172
